**Document Ingestion**

**Installing the dependencies**

In [3]:
!pip install -U -q -r requirements.txt


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


**Importing the dependencies**

In [20]:
import os

from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("punkt_tab")
nltk.download("stopwords")

In [5]:
# print current working directory
!pwd

/Volumes/MyDrive/2_jupyter/jupyter_nb_files/1_youtube/3_courses/code_prep/6_rag/6_3_llamaindex_rag


In [6]:
# configuration
docs_dir_path = "/Volumes/MyDrive/2_jupyter/jupyter_nb_files/1_youtube/3_courses/code_prep/6_rag/6_3_llamaindex_rag/docs_dir"
vector_db_path = "/Volumes/MyDrive/2_jupyter/jupyter_nb_files/1_youtube/3_courses/code_prep/6_rag/6_3_llamaindex_rag/vector_db"
collection_name="documents_collection"

**Document Ingestion**

In [7]:
# define embedding function
embed_model = HuggingFaceEmbedding()

In [8]:
# directory loader
loader = SimpleDirectoryReader(input_dir=docs_dir_path)

In [9]:
# load the documents
documents = loader.load_data()

In [10]:
print(len(documents))

28


In [13]:
# Define persistent DB location
db = chromadb.PersistentClient(path=vector_db_path)

In [18]:
# Create or retrieve the vector collection
chroma_collection = db.get_or_create_collection(name=collection_name)

In [22]:
# Create parser with chunking strategy
parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=50)

In [23]:
# Convert documents to chunks (nodes)
nodes = parser.get_nodes_from_documents(documents)

In [24]:
# Create vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [25]:
# Create storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [26]:
# Create the vector store index
index = VectorStoreIndex(
    nodes, 
    storage_context=storage_context, 
    vector_store=vector_store, 
    embed_model=embed_model
)

In [27]:
print("vector database created")

vector database created
